# Annotating Peaks

In [1]:
import pandas as pd

## Dual Summits Plots

Load files into Pandas dataframes.

In [2]:
enhancer_annotation = pd.read_csv("~/Desktop/Divya/Thesis/enhancers/enhancer_annotation.csv")
chip_dualsummits = pd.read_csv("~/Desktop/Divya/Thesis/enhancers/chip_dualsummits.bed", sep='\t', header=None)

First we will look at intergenics, and then at introns. We extract the peak annotations for each.

In [3]:
anno_col = enhancer_annotation['annotation']  ## extract 'annotation column' 
# Get intron and intergenic indices
intron_indices = anno_col.str.contains("Intron")
exon_indices = anno_col.str.contains("Exon")
intergenic_indices = ~exon_indices & ~intron_indices

In [4]:
# Select only the intron and intergenic peaks
intron_peaks = enhancer_annotation[intron_indices]
intergenic_peaks = enhancer_annotation[intergenic_indices]

In [5]:
# Sanity checks
print("Sanity Checks\n-------------")
print("Total number of rows:", len(enhancer_annotation))
print("Number of Intron rows:", len(intron_peaks))
print("Number of Intergenic rows:", len(intergenic_peaks))
print("Intersections between Intron and Intergenic rows:", (intron_indices & intergenic_indices).sum())

Sanity Checks
-------------
Total number of rows: 6032
Number of Intron rows: 2003
Number of Intergenic rows: 3001
Intersections between Intron and Intergenic rows: 0


### Introns and Intergenics
#### Determine intron and intergenic chip peaks 


In [6]:
intron_chip = chip_dualsummits[chip_dualsummits.iloc[:,3].isin(intron_peaks.iloc[:,5])]
intergenic_chip = chip_dualsummits[chip_dualsummits.iloc[:,3].isin(intergenic_peaks.iloc[:,5])]

In [7]:
print("Number of intronic:", len(intron_chip))
print("Number of intergenic:", len(intergenic_chip))

Number of intronic: 2191
Number of intergenic: 3132


#### Determine intergenic X1 RNA.

Load files.

In [8]:
RNAchipintron = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/rna_dualchipintrons.tab",
                            sep="\t",
                            header=None,
                            skiprows=3)
RNAchipintergenic = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/rna_dualchipintergenic.tab",
                                sep="\t",
                                header=None,
                                skiprows=3)

Filter out relevant peaks.

In [9]:
# Set threshold for x1rna regions
x1rna_threshold = 0
# Compute 'filter' mask
intron_x1rna_mask = RNAchipintron.mean(axis=1).values > x1rna_threshold
intergenic_x1rna_mask = RNAchipintergenic.mean(axis=1).values > x1rna_threshold
# Extract all accessible peaks
intron_x1rna = intron_chip[intron_x1rna_mask]
intergenic_x1rna = intergenic_chip[intergenic_x1rna_mask]

In [10]:
print("Sanity check\n------------")
print("Number of intronic X1 RNA:", len(intron_x1rna))
print("Number of intergenic X1 RNA:", len(intergenic_x1rna))

Sanity check
------------
Number of intronic X1 RNA: 2074
Number of intergenic X1 RNA: 2807


#### Determine accessible regions.
Load files.

In [11]:
intron_atac = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/atacchipintron.tab",
                          sep="\t",
                          header=None,
                          skiprows=3)
intergenic_atac = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/atacchipintergenic.tab",
                              sep="\t",
                              header=None,
                              skiprows=3)

Filter out relevant peaks.

In [12]:
# Set threshold for accessible regions
intron_accessible_threshold = 5
intergenic_accessible_threshold = 5
# Compute 'filter' masks
intron_accessible_mask = intron_atac.mean(axis=1).values >= intron_accessible_threshold
intergenic_accessible_mask = intergenic_atac.mean(axis=1).values >= intergenic_accessible_threshold
# Extract all accessible peaks
intron_accessible = intron_chip[intron_accessible_mask]
intergenic_accessible = intergenic_chip[intergenic_accessible_mask]

In [13]:
print("Number of accessible intronics:", len(intron_accessible))
print("Number of accessible intergenics:", len(intergenic_accessible))

Number of accessible intronics: 1472
Number of accessible intergenics: 2161


#### Analyse H3K4me3 overlap

In [14]:
h3k4me3_values = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/h3k4me3around_dualsummits.tab",
                             sep="\t",
                             header=None,
                             skiprows=3)

In [15]:
# Set threshold for h3k4me3 regions
h3m4ke3_threshold = 0.58
# Compute the 'filter' mask
h3m4ke3_mask = h3k4me3_values.mean(axis=1).values >= h3m4ke3_threshold
# Extract all h3k4me3 peaks
h3k4me3_chip = chip_dualsummits[h3m4ke3_mask]

In [16]:
print("Number of H3K4me3 peaks:", len(h3k4me3_chip))

Number of H3K4me3 peaks: 2122


#### Collate data for UpSetR figure
Load externally generated files for X1 ATAC.

In [17]:
# Load externally generated files
intron_x1_chip = pd.read_csv("~/Desktop/Divya/Thesis/enhancers/chip_intron_x1atac.bed",
                             sep="\t",
                             header=None)
intergenic_x1_chip = pd.read_csv("~/Desktop/Divya/Thesis/enhancers/chip_intergenic_x1atac.bed",
                                 sep="\t",
                                 header=None)

Determine peak sets from all subsets generated so far..

In [18]:
# Compute peak sets from all subsets
intron_peakset = set(intron_chip.iloc[:,3])
intergenic_peakset = set(intergenic_chip.iloc[:,3])

intron_accessible_peakset = set(intron_accessible.iloc[:,3])
intergenic_accessible_peakset = set(intergenic_accessible.iloc[:,3])
accessible_peakset = intron_accessible_peakset.union(intergenic_accessible_peakset)

intron_x1_peakset = set(intron_x1_chip.iloc[:,3])
intergenic_x1_peakset = set(intergenic_x1_chip.iloc[:,3])
x1_peakset = intron_x1_peakset.union(intergenic_x1_peakset)

# Abandoned: x1rna_peakset = set(intron_x1rna.iloc[:,3]).union(set(intergenic_x1rna.iloc[:,3]))
x1rna_peakset = set(intergenic_x1rna.iloc[:,3])

h3k4me3_peakset = set(h3k4me3_chip.iloc[:,3])


In [19]:
# Sanity check output
print("Peakset sizes:\n--------------")
print("Intronic:", len(intron_peakset))
print("Intergenic:", len(intergenic_peakset))
print("Accessible:", len(accessible_peakset))
print("X1ATAC:", len(x1_peakset))
print("X1RNA:", len(x1rna_peakset))
print("H3K4me3:", len(h3k4me3_peakset))

Peakset sizes:
--------------
Intronic: 1640
Intergenic: 2350
Accessible: 2656
X1ATAC: 172
X1RNA: 2126
H3K4me3: 1633


Create new dictionary that helps us generate the UpSetR command.

In [20]:
d = {}

# OPTIONAL:
# d['Intronic region'] = len(intron_peakset)
# d['Intergenic region'] = len(intergenic_peakset)
# d['Accessible region'] = len(accessible_peakset)
# d['ATAC peak'] = len(x1_peakset)
# d['RNA transcript'] = len(x1rna_peakset)
# d['H3K4me3'] = len(h3k4me3_peakset)

d['Intronic region&ATAC peak'] = len(intron_peakset.intersection(x1_peakset))  # or should this be intron_x1_peakset???
d['Intergenic region&ATAC peak'] = len(intergenic_peakset.intersection(x1_peakset))

d['Intronic region&H3K4me3'] = len(intron_peakset.intersection(h3k4me3_peakset))
d['Intergenic region&H3K4me3'] = len(intergenic_peakset.intersection(h3k4me3_peakset))

d['Intergenic region&ATAC peak&RNA transcript'] = len(intergenic_peakset.intersection(x1_peakset, x1rna_peakset))

d['Intronic region&Accessible region'] = len(intron_peakset.intersection(accessible_peakset))
d['Intergenic region&Accessible region'] = len(intergenic_peakset.intersection(accessible_peakset))

d['Intergenic region&ATAC peak&RNA transcript&H3K4me3'] = len(intergenic_peakset.intersection(x1_peakset,
                                                                           x1rna_peakset,
                                                                           h3k4me3_peakset))

d['Intronic region&Accessible region&H3K4me3'] = len(intron_peakset.intersection(accessible_peakset,
                                                                   h3k4me3_peakset))
d['Intergenic region&Accessible region&H3K4me3'] = len(intergenic_peakset.intersection(accessible_peakset,
                                                                         h3k4me3_peakset))

d['Intronic region&ATAC peak&H3K4me3'] = len(intron_peakset.intersection(x1_peakset,
                                                               h3k4me3_peakset))
d['Intergenic region&ATAC peak&H3K4me3'] = len(intergenic_peakset.intersection(x1_peakset,
                                                                     h3k4me3_peakset))

d['Intergenic region&RNA transcript'] = len(intergenic_peakset.intersection(x1rna_peakset))

d['Intergenic region&Accessible region&RNA transcript'] = len(intergenic_peakset.intersection(accessible_peakset, x1rna_peakset))


In [21]:
sets = ["Intronic region", "Intergenic region", "Accessible region", "ATAC peak", "RNA transcript", "H3K4me3"]

Define helper function that outputs the upSetR command given a data dictionary.

In [22]:
def upSetR_cmd(d, sets, filename, colour="gray23"):
    """Computes the R commands required to plot the data in dictionary d using the upSetR library."""
    vector = "c(\n" + ',\n'.join([f"`{key}` = {d[key]}" for key in d]) + ")"
    sets = 'c(' + ', '.join([f"\'{s}\'" for s in sets]) + ")"
    output = "library(UpSetR)\n"
    output += f"expressionInput <- {vector}\n"
    output += "data <- fromExpression(expressionInput)\n"
    output += f"sets <- {sets}\n"
    output += f"pdf(file=\"{filename}\")\n"
    output += f'upset(data, sets=sets,\n order.by=c("degree", "freq"),\n main.bar.color=\"{colour}\",\n matrix.color=\"{colour}\",\n text.scale=1.3)\n'
    output += "dev.off()\n"
    return output

In [23]:
print(upSetR_cmd(d, sets, "dualsummits.pdf"))

library(UpSetR)
expressionInput <- c(
`Intronic region&ATAC peak` = 54,
`Intergenic region&ATAC peak` = 121,
`Intronic region&H3K4me3` = 350,
`Intergenic region&H3K4me3` = 1074,
`Intergenic region&ATAC peak&RNA transcript` = 118,
`Intronic region&Accessible region` = 1085,
`Intergenic region&Accessible region` = 1605,
`Intergenic region&ATAC peak&RNA transcript&H3K4me3` = 66,
`Intronic region&Accessible region&H3K4me3` = 281,
`Intergenic region&Accessible region&H3K4me3` = 867,
`Intronic region&ATAC peak&H3K4me3` = 12,
`Intergenic region&ATAC peak&H3K4me3` = 67,
`Intergenic region&RNA transcript` = 2126,
`Intergenic region&Accessible region&RNA transcript` = 1505)
data <- fromExpression(expressionInput)
sets <- c('Intronic region', 'Intergenic region', 'Accessible region', 'ATAC peak', 'RNA transcript', 'H3K4me3')
pdf(file="dualsummits.pdf")
upset(data, sets=sets,
 order.by=c("degree", "freq"),
 main.bar.color="gray23",
 matrix.color="gray23",
 text.scale=1.3)
dev.off()



## Generate output file

In [24]:
output_df = pd.read_csv("~/Desktop/Divya/Thesis/enhancers/enhancer_annotation.csv")
output_df['Type'] = 'chip'
output_df['Region'] = None
output_df['ATAC peak'] = 0
output_df['RNA transcript'] = 0
output_df['H3K4me3'] = 0
output_df['Accessible region'] = 0

for i in range(len(output_df)):
    peakname = output_df.iloc[i,5]
    if peakname in intron_peakset: output_df.iloc[i,21] = "Intronic"
    if peakname in intergenic_peakset: output_df.iloc[i,21] = "Intergenic"
    if peakname in x1_peakset: output_df.iloc[i,22] = 1
    if peakname in x1rna_peakset: output_df.iloc[i,23] = 1
    if peakname in h3k4me3_peakset: output_df.iloc[i,24] = 1
    if peakname in accessible_peakset: output_df.iloc[i,25] = 1

In [25]:
output_df.to_csv("~/Desktop/Divya/Thesis/enhancers/enhancer_annotated.csv", index=False)

## Single H3K4me1 (not part of the dual summits)

In [26]:
h3k4me1_summits = pd.read_csv("~/Desktop/Divya/Thesis/enhancers/h3k4me1_summits.bed", sep='\t', header=None)

In [27]:
# Compute the difference of h3k27ac_summits and dual_k4_summits
chip_k4_dualsummits = pd.read_csv("~/Desktop/Divya/Thesis/enhancers/chip_dualsummits.bed", sep='\t', header=None)
dual_peak_names = chip_k4_dualsummits.iloc[:,3]
unique_k4_chip = h3k4me1_summits[~h3k4me1_summits.iloc[:,3].isin(dual_peak_names)]

In [28]:
enhancer_annotation_k4 = pd.read_csv("~/Desktop/Divya/Thesis/enhancers/enhancer_annotation_h3k4me1.csv")
anno_col_k4 = enhancer_annotation_k4['annotation']  ## extract 'annotation column' 
# Get intron and intergenic indices
intron_indices_k4 = anno_col_k4.str.contains("Intron")
exon_indices_k4 = anno_col_k4.str.contains("Exon")
intergenic_indices_k4 = ~exon_indices_k4 & ~intron_indices_k4
# Select only the intron and intergenic peaks
intron_peaks_k4 = enhancer_annotation_k4[intron_indices_k4]
intergenic_peaks_k4 = enhancer_annotation_k4[intergenic_indices_k4]

#### Determine intronic and intergenic peaks

In [29]:
intron_k4_chip = unique_k4_chip[unique_k4_chip.iloc[:,3].isin(intron_peaks_k4.iloc[:,5])]
intergenic_k4_chip = unique_k4_chip[unique_k4_chip.iloc[:,3].isin(intergenic_peaks_k4.iloc[:,5])]

In [30]:
print("Number of intron:", len(intron_k4_chip))
print("Number of intergenic:", len(intergenic_k4_chip))

Number of intron: 3990
Number of intergenic: 4043


#### Determine intron and intergenic X1 RNA.

In [31]:
RNAchipintron_k4 = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/rna_intron_unique_h3k4me1.tab",
                            sep="\t",
                            header=None,
                            skiprows=3)
RNAchipintergenic_k4 = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/rna_intergenic_unique_h3k4me1.tab",
                                sep="\t",
                                header=None,
                                skiprows=3)

Filter out relevant peaks.

In [32]:
# Set threshold for x1rna regions
x1rna_threshold_k4 = 0
# Compute 'filter' mask
intron_x1rna_mask_k4 = RNAchipintron_k4.mean(axis=1).values > x1rna_threshold_k4
intergenic_x1rna_mask_k4 = RNAchipintergenic_k4.mean(axis=1).values > x1rna_threshold_k4
# Extract all accessible peaks
intron_x1rna_k4 = intron_k4_chip[intron_x1rna_mask_k4]
intergenic_x1rna_k4 = intergenic_k4_chip[intergenic_x1rna_mask_k4]

In [33]:
print("Number of intron X1 RNA:", len(intron_x1rna_k4))
print("Number of intergenic X1 RNA:", len(intergenic_x1rna_k4))

Number of intron X1 RNA: 3805
Number of intergenic X1 RNA: 3800


#### Determine accessible regions.

In [34]:
intron_atac_tab_k4 = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/atac_intron_unique_h3k4me1.tab",
                          sep="\t",
                          header=None,
                          skiprows=3)
intergenic_atac_tab_k4 = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/atac_intergenic_unique_h3k4me1.tab",
                              sep="\t",
                              header=None,
                              skiprows=3)

In [35]:
# Set threshold for accessible regions
intron_accessible_threshold_k4 = 5
intergenic_accessible_threshold_k4 = 5
# Compute 'filter' masks
intron_accessible_mask_k4 = intron_atac_tab_k4.mean(axis=1).values >= intron_accessible_threshold_k4
intergenic_accessible_mask_k4 = intergenic_atac_tab_k4.mean(axis=1).values >= intergenic_accessible_threshold_k4
# Extract all accessible peaks
intron_accessible_k4 = intron_k4_chip[intron_accessible_mask_k4]
intergenic_accessible_k4 = intergenic_k4_chip[intergenic_accessible_mask_k4]

In [36]:
print("Number of accessible intronics:", len(intron_accessible_k4))
print("Number of accessible intergenics:", len(intergenic_accessible_k4))

Number of accessible intronics: 2279
Number of accessible intergenics: 2649


#### Analyse H3K4me3 overlap

In [37]:
file_intron = "/Users/edwinlock/Desktop/Divya/Thesis/enhancers/k4me3around_unique_h3k4me1_introns.tab"
file_intergenic = "/Users/edwinlock/Desktop/Divya/Thesis/enhancers/k4me3around_unique_h3k4me1_intergenic.tab"
intron_h3k4me3_around_h3k4me1_values = pd.read_csv(file_intron,
                                                   sep="\t",
                                                   header=None,
                                                   skiprows=3)
intergenic_h3k4me3_around_h3k4me1_values = pd.read_csv(file_intergenic,
                                                       sep="\t",
                                                       header=None,
                                                       skiprows=3)

In [38]:
print("Mean H3K4me3 values\n-------------------")
print("Intronic:", intron_h3k4me3_around_h3k4me1_values.mean(axis=1).mean())
print("Intergenic:", intergenic_h3k4me3_around_h3k4me1_values.mean(axis=1).mean())

Mean H3K4me3 values
-------------------
Intronic: 0.4527863002576341
Intergenic: 0.8604562046594212


In [39]:
# Set threshold for h3k4me3 regions
h3m4ke3_intron_threshold_k4 = 0.45
h3m4ke3_intergenic_threshold_k4 = 0.86
# Compute the 'filter' masks
h3m4ke3_intron_mask_k4 = (intron_h3k4me3_around_h3k4me1_values.mean(axis=1).values >=
                          h3m4ke3_intron_threshold_k4)
h3m4ke3_intergenic_mask_k4 = (intergenic_h3k4me3_around_h3k4me1_values.mean(axis=1).values >=
                              h3m4ke3_intergenic_threshold_k4)
# Extract all h3k4me3 peaks
intron_h3k4me3_k4_chip = intron_k4_chip[h3m4ke3_intron_mask_k4]
intergenic_h3k4me3_k4_chip = intergenic_k4_chip[h3m4ke3_intergenic_mask_k4]

In [40]:
print("Number of H3K4me3 peaks\n-----------------------")
print("Intronic:", len(intron_h3k4me3_k4_chip))
print("Intergenic:", len(intergenic_h3k4me3_k4_chip))

Number of H3K4me3 peaks
-----------------------
Intronic: 1054
Intergenic: 1618


#### Assemble data for UpSetR command

In [41]:
# Load externally generated files
intron_atac_k4 = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/intron_unique_h3k4me1_x1atac.bed",
                          sep="\t",
                          header=None,
                          skiprows=3)
intergenic_atac_k4 = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/intergenic_unique_h3k4me1_x1atac.bed",
                              sep="\t",
                              header=None,
                              skiprows=3)

In [42]:
# Compute peak sets from all subsets
intron_peakset_k4 = set(intron_k4_chip.iloc[:,3])
intergenic_peakset_k4 = set(intergenic_k4_chip.iloc[:,3])

intron_accessible_peakset_k4 = set(intron_accessible_k4.iloc[:,3])
intergenic_accessible_peakset_k4 = set(intergenic_accessible_k4.iloc[:,3])
accessible_peakset_k4 = intron_accessible_peakset_k4.union(intergenic_accessible_peakset_k4)

intron_x1_peakset_k4 = set(intron_atac_k4.iloc[:,3])
intergenic_x1_peakset_k4 = set(intergenic_atac_k4.iloc[:,3])
x1_peakset_k4 = intron_x1_peakset_k4.union(intergenic_x1_peakset_k4)

# x1rna_peakset_k4 = set(intron_x1rna_k4.iloc[:,3]).union(set(intergenic_x1rna_k4.iloc[:,3]))
x1rna_peakset_k4 = set(intergenic_x1rna_k4.iloc[:,3])

h3k4me3_peakset_intron_k4 = set(intron_h3k4me3_k4_chip.iloc[:,3])
h3k4me3_peakset_intergenic_k4 = set(intergenic_h3k4me3_k4_chip.iloc[:,3])
h3k4me3_peakset_k4 = h3k4me3_peakset_intron_k4.union(h3k4me3_peakset_intergenic_k4)

In [43]:
# Sanity check
print("Peakset sizes:\n--------------")
print("Intronic:", len(intron_peakset_k4))
print("Intergenic:", len(intergenic_peakset_k4))
print("Accessible:", len(accessible_peakset_k4))
print("X1ATAC:", len(x1_peakset_k4))
print("X1RNA:", len(x1rna_peakset_k4))
print("H3K4me3:", len(h3k4me3_peakset_k4))

Peakset sizes:
--------------
Intronic: 3990
Intergenic: 4043
Accessible: 4928
X1ATAC: 325
X1RNA: 3800
H3K4me3: 2672


Create new dictionary that helps us generate the UpSetR command.

In [44]:
d_k4 = {}

# OPTIONAL:
# d_k4['Intronic region'] = len(intron_peakset)
# d_k4['Intergenic region'] = len(intergenic_peakset)
# d_k4['Accessible region'] = len(accessible_peakset)
# d_k4['ATAC peak'] = len(x1_peakset)
# d_k4['RNA transcript'] = len(x1rna_peakset)
# d_k4['H3K4me3'] = len(h3k4me3_peakset)

d_k4['Intronic region&ATAC peak'] = len(intron_peakset_k4.intersection(x1_peakset_k4))  # or should this be intron_x1_peakset???
d_k4['Intergenic region&ATAC peak'] = len(intergenic_peakset_k4.intersection(x1_peakset_k4))

d_k4['Intronic region&H3K4me3'] = len(intron_peakset_k4.intersection(h3k4me3_peakset_k4))
d_k4['Intergenic region&H3K4me3'] = len(intergenic_peakset_k4.intersection(h3k4me3_peakset_k4))

d_k4['Intergenic region&ATAC peak&RNA transcript'] = len(intergenic_peakset_k4.intersection(x1_peakset_k4, x1rna_peakset_k4))

d_k4['Intronic region&Accessible region'] = len(intron_peakset_k4.intersection(accessible_peakset_k4))
d_k4['Intergenic region&Accessible region'] = len(intergenic_peakset_k4.intersection(accessible_peakset_k4))

d_k4['Intergenic region&ATAC peak&RNA transcript&H3K4me3'] = len(intergenic_peakset_k4.intersection(x1_peakset_k4,
                                                                           x1rna_peakset_k4,
                                                                           h3k4me3_peakset_k4))

d_k4['Intronic region&Accessible region&H3K4me3'] = len(intron_peakset_k4.intersection(accessible_peakset_k4,
                                                                   h3k4me3_peakset_k4))
d_k4['Intergenic region&Accessible region&H3K4me3'] = len(intergenic_peakset_k4.intersection(accessible_peakset_k4,
                                                                         h3k4me3_peakset_k4))

d_k4['Intronic region&ATAC peak&H3K4me3'] = len(intron_peakset_k4.intersection(x1_peakset_k4,
                                                               h3k4me3_peakset_k4))
d_k4['Intergenic region&ATAC peak&H3K4me3'] = len(intergenic_peakset_k4.intersection(x1_peakset_k4,
                                                                     h3k4me3_peakset_k4))

d_k4['Intergenic region&RNA transcript'] = len(intergenic_peakset_k4.intersection(x1rna_peakset_k4))

d_k4['Intergenic region&Accessible region&RNA transcript'] = len(intergenic_peakset_k4.intersection(accessible_peakset_k4, x1rna_peakset_k4))


In [45]:
sets_k4 = ["Intronic region", "Intergenic region", "Accessible region", "ATAC peak", "RNA transcript", "H3K4me3"]

In [46]:
print(upSetR_cmd(d_k4, sets_k4, "h3k4me1.pdf", colour="slateblue4"))

library(UpSetR)
expressionInput <- c(
`Intronic region&ATAC peak` = 118,
`Intergenic region&ATAC peak` = 207,
`Intronic region&H3K4me3` = 1054,
`Intergenic region&H3K4me3` = 1618,
`Intergenic region&ATAC peak&RNA transcript` = 197,
`Intronic region&Accessible region` = 2279,
`Intergenic region&Accessible region` = 2649,
`Intergenic region&ATAC peak&RNA transcript&H3K4me3` = 53,
`Intronic region&Accessible region&H3K4me3` = 704,
`Intergenic region&Accessible region&H3K4me3` = 1152,
`Intronic region&ATAC peak&H3K4me3` = 33,
`Intergenic region&ATAC peak&H3K4me3` = 54,
`Intergenic region&RNA transcript` = 3800,
`Intergenic region&Accessible region&RNA transcript` = 2561)
data <- fromExpression(expressionInput)
sets <- c('Intronic region', 'Intergenic region', 'Accessible region', 'ATAC peak', 'RNA transcript', 'H3K4me3')
pdf(file="h3k4me1.pdf")
upset(data, sets=sets,
 order.by=c("degree", "freq"),
 main.bar.color="slateblue4",
 matrix.color="slateblue4",
 text.scale=1.3)
dev.off()



## Generate output file

In [47]:
k4_output_df = pd.read_csv("~/Desktop/Divya/Thesis/enhancers/enhancer_annotation_h3k4me1.csv")
k4_output_df['Type'] = 'k4me1'
k4_output_df['Region'] = None
k4_output_df['ATAC peak'] = 0
k4_output_df['RNA transcript'] = 0
k4_output_df['H3K4me3'] = 0
k4_output_df['Accessible region'] = 0

for i in range(len(k4_output_df)):
    peakname = k4_output_df.iloc[i,5]
    if peakname in intron_peakset_k4: k4_output_df.iloc[i,21] = "Intronic"
    if peakname in intergenic_peakset_k4: k4_output_df.iloc[i,21] = "Intergenic"
    if peakname in x1_peakset_k4: k4_output_df.iloc[i,22] = 1
    if peakname in x1rna_peakset_k4: k4_output_df.iloc[i,23] = 1
    if peakname in h3k4me3_peakset_k4: k4_output_df.iloc[i,24] = 1
    if peakname in accessible_peakset_k4: k4_output_df.iloc[i,25] = 1

In [48]:
k4_output_df.to_csv("~/Desktop/Divya/Thesis/enhancers/enhancer_h3k4me3_annotated.csv", index=False)

## Single H3K27ac (not part of the dual summits)

In [49]:
h3k27ac_summits = pd.read_csv("~/Desktop/Divya/Thesis/enhancers/h3k27ac_summits.bed", sep='\t', header=None)

In [50]:
chip_k27_dualsummits = pd.read_csv("~/Desktop/Divya/Thesis/enhancers/chip_dualsummits_k27ac_id.bed", sep='\t', header=None)
dual_peak_names = chip_k27_dualsummits.iloc[:,3]
unique_k27_chip = h3k27ac_summits[~h3k27ac_summits.iloc[:,3].isin(dual_peak_names)]

In [51]:
# Save to file
unique_k27_chip.to_csv("~/Desktop/Divya/Thesis/enhancers/chip_k27_summits.bed", sep="\t", header=None)

In [52]:
enhancer_annotation_k27 = pd.read_csv("~/Desktop/Divya/Thesis/enhancers/enhancer_annotation_h3k27ac.csv")
anno_col_k27 = enhancer_annotation_k27['annotation']  ## extract 'annotation column' 
# Get intron and intergenic indices
intron_indices_k27 = anno_col_k27.str.contains("Intron")
exon_indices_k27 = anno_col_k27.str.contains("Exon")
intergenic_indices_k27 = ~exon_indices_k27 & ~intron_indices_k27
# Select only the intron and intergenic peaks
intron_peaks_k27 = enhancer_annotation_k27[intron_indices_k27]
intergenic_peaks_k27 = enhancer_annotation_k27[intergenic_indices_k27]

#### Determine intronic and intergenic peaks

In [53]:
intron_k27_chip = unique_k27_chip[unique_k27_chip.iloc[:,3].isin(intron_peaks_k27.iloc[:,5])]
intergenic_k27_chip = unique_k27_chip[unique_k27_chip.iloc[:,3].isin(intergenic_peaks_k27.iloc[:,5])]

In [54]:
print("Number of intron:", len(intron_k27_chip))
print("Number of intergenic:", len(intergenic_k27_chip))

Number of intron: 12005
Number of intergenic: 19925


#### Determine intron and intergenic X1 RNA.

In [55]:
RNAchipintron_k27 = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/rna_intron_unique_h3k27ac.tab",
                            sep="\t",
                            header=None,
                            skiprows=3)
RNAchipintergenic_k27 = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/rna_intergenic_unique_h3k27ac.tab",
                                sep="\t",
                                header=None,
                                skiprows=3)

In [56]:
# Set threshold for x1rna regions
x1rna_threshold_k27 = 5
# Compute 'filter' mask
intron_x1rna_mask_k27 = RNAchipintron_k27.mean(axis=1).values > x1rna_threshold_k27
intergenic_x1rna_mask_k27 = RNAchipintergenic_k27.mean(axis=1).values > x1rna_threshold_k27
# Extract all accessible peaks
intron_x1rna_k27 = intron_k27_chip[intron_x1rna_mask_k27]
intergenic_x1rna_k27 = intergenic_k27_chip[intergenic_x1rna_mask_k27]

In [57]:
print("Number of intron X1 RNA:", len(intron_x1rna_k27))
print("Number of intergenic X1 RNA:", len(intergenic_x1rna_k27))

Number of intron X1 RNA: 182
Number of intergenic X1 RNA: 2074


#### Determine accessible regions.

In [58]:
intron_atac_tab_k27 = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/atac_intron_unique_h3k27ac.tab",
                          sep="\t",
                          header=None,
                          skiprows=3)
intergenic_atac_tab_k27 = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/atac_intergenic_unique_h3k27ac.tab",
                              sep="\t",
                              header=None,
                              skiprows=3)

In [59]:
# Set threshold for accessible regions
intron_accessible_threshold_k27 = 5
intergenic_accessible_threshold_k27 = 5
# Compute 'filter' masks
intron_accessible_mask_k27 = intron_atac_tab_k27.mean(axis=1).values >= intron_accessible_threshold_k27
intergenic_accessible_mask_k27 = intergenic_atac_tab_k27.mean(axis=1).values >= intergenic_accessible_threshold_k27
# Extract all accessible peaks
intron_accessible_k27 = intron_k27_chip[intron_accessible_mask_k27]
intergenic_accessible_k27 = intergenic_k27_chip[intergenic_accessible_mask_k27]

In [60]:
print("Number of accessible intronics:", len(intron_accessible_k27))
print("Number of accessible intergenics:", len(intergenic_accessible_k27))

Number of accessible intronics: 5153
Number of accessible intergenics: 11010


#### Analyse H3K4me3 region overlap.

In [61]:
file_intron = "/Users/edwinlock/Desktop/Divya/Thesis/enhancers/k4me3around_unique_h3k27ac_introns.tab"
file_intergenic = "/Users/edwinlock/Desktop/Divya/Thesis/enhancers/k4me3around_unique_h3k27ac_intergenic.tab"
intron_h3k4me3_around_h3k27ac_values = pd.read_csv(file_intron,
                                                   sep="\t",
                                                   header=None,
                                                   skiprows=3)
intergenic_h3k4me3_around_h3k27ac_values = pd.read_csv(file_intergenic,
                                                       sep="\t",
                                                       header=None,
                                                       skiprows=3)

In [62]:
print("Mean H3K4me3 values\n-------------------")
print("Intronic:", intron_h3k4me3_around_h3k27ac_values.mean(axis=1).mean())
print("Intergenic:", intergenic_h3k4me3_around_h3k27ac_values.mean(axis=1).mean())

Mean H3K4me3 values
-------------------
Intronic: 0.0007394901849238641
Intergenic: 0.3724214907946551


In [63]:
# Set threshold for h3k4me3 regions
h3m4ke3_intron_threshold_k27 = 0
h3m4ke3_intergenic_threshold_k27 = 0.372
# Compute the 'filter' masks
h3m4ke3_intron_mask_k27 = (intron_h3k4me3_around_h3k27ac_values.mean(axis=1).values >=
                          h3m4ke3_intron_threshold_k27)
h3m4ke3_intergenic_mask_k27 = (intergenic_h3k4me3_around_h3k27ac_values.mean(axis=1).values >=
                              h3m4ke3_intergenic_threshold_k27)
# Extract all h3k4me3 peaks
intron_h3k4me3_k27_chip = intron_k27_chip[h3m4ke3_intron_mask_k27]
intergenic_h3k4me3_k27_chip = intergenic_k27_chip[h3m4ke3_intergenic_mask_k27]

In [64]:
print("Number of H3K4me3 peaks\n-----------------------")
print("Intronic:", len(intron_h3k4me3_k27_chip))
print("Intergenic:", len(intergenic_h3k4me3_k27_chip))

Number of H3K4me3 peaks
-----------------------
Intronic: 4995
Intergenic: 3967


#### Assemble final file

In [65]:
# Load externally generated files
intron_atac_k27 = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/intron_unique_h3k27ac_x1atac.bed",
                          sep="\t",
                          header=None,
                          skiprows=3)
intergenic_atac_k27 = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/intergenic_unique_h3k27ac_x1atac.bed",
                              sep="\t",
                              header=None,
                              skiprows=3)

In [66]:
# Compute peak sets from all subsets
intron_peakset_k27 = set(intron_k27_chip.iloc[:,3])
intergenic_peakset_k27 = set(intergenic_k27_chip.iloc[:,3])

intron_accessible_peakset_k27 = set(intron_accessible_k27.iloc[:,3])
intergenic_accessible_peakset_k27 = set(intergenic_accessible_k27.iloc[:,3])
accessible_peakset_k27 = intron_accessible_peakset_k27.union(intergenic_accessible_peakset_k27)

intron_x1_peakset_k27 = set(intron_atac_k27.iloc[:,3])
intergenic_x1_peakset_k27 = set(intergenic_atac_k27.iloc[:,3])
x1_peakset_k27 = intron_x1_peakset_k27.union(intergenic_x1_peakset_k27)

# x1rna_peakset_k27 = set(intron_x1rna_k27.iloc[:,3]).union(set(intergenic_x1rna_k27.iloc[:,3]))
x1rna_peakset_k27 = set(intergenic_x1rna_k27.iloc[:,3])

h3k4me3_peakset_intron_k27 = set(intron_h3k4me3_k27_chip.iloc[:,3])
h3k4me3_peakset_intergenic_k27 = set(intergenic_h3k4me3_k27_chip.iloc[:,3])
h3k4me3_peakset_k27 = h3k4me3_peakset_intron_k27.union(h3k4me3_peakset_intergenic_k27)

In [67]:
print("Peakset sizes:\n--------------")
print("Intronic:", len(intron_peakset_k27))
print("Intergenic:", len(intergenic_peakset_k27))
print("Accessible:", len(accessible_peakset_k27))
print("X1ATAC:", len(x1_peakset_k27))
print("X1RNA:", len(x1rna_peakset_k27))
print("H3K4me3:", len(h3k4me3_peakset_k27))


Peakset sizes:
--------------
Intronic: 12005
Intergenic: 19925
Accessible: 16163
X1ATAC: 899
X1RNA: 2074
H3K4me3: 8962


In [68]:
d_k27 = {}

# OPTIONAL:
# d_k27['Intronic region'] = len(intron_peakset)
# d_k27['Intergenic region'] = len(intergenic_peakset)
# d_k27['Accessible region'] = len(accessible_peakset)
# d_k27['ATAC peak'] = len(x1_peakset)
# d_k27['RNA transcript'] = len(x1rna_peakset)
# d_k27['H3K4me3'] = len(h3k4me3_peakset)

d_k27['Intronic region&ATAC peak'] = len(intron_peakset_k27.intersection(x1_peakset_k27))  # or should this be intron_x1_peakset???
d_k27['Intergenic region&ATAC peak'] = len(intergenic_peakset_k27.intersection(x1_peakset_k27))

d_k27['Intronic region&H3K4me3'] = len(intron_peakset_k27.intersection(h3k4me3_peakset_k27))
d_k27['Intergenic region&H3K4me3'] = len(intergenic_peakset_k27.intersection(h3k4me3_peakset_k27))

d_k27['Intergenic region&ATAC peak&RNA transcript'] = len(intergenic_peakset_k27.intersection(x1_peakset_k27, x1rna_peakset_k27))

d_k27['Intronic region&Accessible region'] = len(intron_peakset_k27.intersection(accessible_peakset_k27))
d_k27['Intergenic region&Accessible region'] = len(intergenic_peakset_k27.intersection(accessible_peakset_k27))

d_k27['Intergenic region&ATAC peak&RNA transcript&H3K4me3'] = len(intergenic_peakset_k27.intersection(x1_peakset_k27,
                                                                           x1rna_peakset_k27,
                                                                           h3k4me3_peakset_k27))

d_k27['Intronic region&Accessible region&H3K4me3'] = len(intron_peakset_k27.intersection(accessible_peakset_k27,
                                                                   h3k4me3_peakset_k27))
d_k27['Intergenic region&Accessible region&H3K4me3'] = len(intergenic_peakset_k27.intersection(accessible_peakset_k27,
                                                                         h3k4me3_peakset_k27))

d_k27['Intronic region&ATAC peak&H3K4me3'] = len(intron_peakset_k27.intersection(x1_peakset_k27,
                                                               h3k4me3_peakset_k27))
d_k27['Intergenic region&ATAC peak&H3K4me3'] = len(intergenic_peakset_k27.intersection(x1_peakset_k27,
                                                                     h3k4me3_peakset_k27))

d_k27['Intergenic region&RNA transcript'] = len(intergenic_peakset_k27.intersection(x1rna_peakset_k27))

d_k27['Intergenic region&Accessible region&RNA transcript'] = len(intergenic_peakset_k27.intersection(accessible_peakset_k27, x1rna_peakset_k27))


In [69]:
sets_k27 = ["Intronic region", "Intergenic region", "Accessible region", "ATAC peak", "RNA transcript", "H3K4me3"]

In [70]:
print(upSetR_cmd(d_k27, sets_k27, "unique_h3k27ac.pdf", colour="red4"))

library(UpSetR)
expressionInput <- c(
`Intronic region&ATAC peak` = 257,
`Intergenic region&ATAC peak` = 642,
`Intronic region&H3K4me3` = 4995,
`Intergenic region&H3K4me3` = 3967,
`Intergenic region&ATAC peak&RNA transcript` = 94,
`Intronic region&Accessible region` = 5153,
`Intergenic region&Accessible region` = 11010,
`Intergenic region&ATAC peak&RNA transcript&H3K4me3` = 72,
`Intronic region&Accessible region&H3K4me3` = 2394,
`Intergenic region&Accessible region&H3K4me3` = 2599,
`Intronic region&ATAC peak&H3K4me3` = 104,
`Intergenic region&ATAC peak&H3K4me3` = 143,
`Intergenic region&RNA transcript` = 2074,
`Intergenic region&Accessible region&RNA transcript` = 1497)
data <- fromExpression(expressionInput)
sets <- c('Intronic region', 'Intergenic region', 'Accessible region', 'ATAC peak', 'RNA transcript', 'H3K4me3')
pdf(file="unique_h3k27ac.pdf")
upset(data, sets=sets,
 order.by=c("degree", "freq"),
 main.bar.color="red4",
 matrix.color="red4",
 text.scale=1.3)
dev.off()



## Generate output file

In [71]:
k27_output_df = pd.read_csv("~/Desktop/Divya/Thesis/enhancers/enhancer_annotation_h3k27ac.csv")
k27_output_df['Type'] = 'k27ac'
k27_output_df['Region'] = None
k27_output_df['ATAC peak'] = 0
k27_output_df['RNA transcript'] = 0
k27_output_df['H3K4me3'] = 0
k27_output_df['Accessible region'] = 0

for i in range(len(k27_output_df)):
    peakname = k27_output_df.iloc[i,5]
    if peakname in intron_peakset_k27: k27_output_df.iloc[i,17] = "Intronic"
    if peakname in intergenic_peakset_k27: k27_output_df.iloc[i,17] = "Intergenic"
    if peakname in x1_peakset_k27: k27_output_df.iloc[i,18] = 1
    if peakname in x1rna_peakset_k27: k27_output_df.iloc[i,19] = 1
    if peakname in h3k4me3_peakset_k27: k27_output_df.iloc[i,20] = 1
    if peakname in accessible_peakset_k27: k27_output_df.iloc[i,21] = 1

In [72]:
k27_output_df.to_csv("~/Desktop/Divya/Thesis/enhancers/enhancer_h3k27ac_annotated.csv")

## Add FACS annotation

In [73]:
merged_output = pd.concat([output_df, k4_output_df, k27_output_df], sort=False)

In [74]:
merged_output.to_csv("~/Desktop/Divya/Thesis/enhancers/enhancer_annotated_table.csv")

In [75]:
output_df_FACS = merged_output.copy()

In [76]:
output_df_FACS["X1.prop"] = 0
output_df_FACS["X1"] = 0

In [77]:
FACS_prop = pd.read_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/FACS_prop.csv")
FACS_prop

,gene,X1,X2,Xins,total,X1.prop,X2.prop,Xins.prop,Rink.ID
0,MSTRG.1,36.191056,25.270065,54.627823,116.088943,0.311753,0.217678,0.470569,SMESG000026673.1
1,MSTRG.10,1.213314,1.256480,0.385004,2.854797,0.425009,0.440129,0.134862,MSTRG.10
2,MSTRG.100,13.166939,22.418585,39.262879,74.848402,0.175915,0.299520,0.524565,SMESG000026500.1
3,MSTRG.1000,11.094835,6.709955,6.726305,24.531095,0.452276,0.273529,0.274195,SMESG000000245.1
4,MSTRG.10000,18.631952,49.453499,49.371953,117.457403,0.158627,0.421033,0.420339,SMESG000030021.1
...,...,...,...,...,...,...,...,...,...
27998,SMESG000081816.1,0.060211,0.128577,0.078985,0.267773,0.224859,0.480171,0.294970,SMESG000081816.1
27999,SMESG000081849.1,0.100407,0.144069,0.000000,0.244476,0.410704,0.589296,0.000000,SMESG000081849.1
28000,SMESG000081971.1,0.018778,0.057717,0.022917,0.099412,0.188893,0.580580,0.230527,SMESG000081971.1
28001,SMESG000081986.1,0.216758,0.360742,0.352566,0.930067,0.233057,0.387867,0.379076,SMESG000081986.1


In [78]:
output_df_FACS.iloc[1]

seqnames                       dd_Smes_g4_1
start                                780974
end                                  780974
width                                     1
strand                                    *
V4                   k4me1_pooled_1_peak_15
V5                                       33
V6                                        .
V7                                  2.75169
V8                                  6.38865
V9                                  3.32508
annotation                Distal Intergenic
geneChr                                   1
geneStart                            801361
geneEnd                              840553
geneLength                            39193
geneStrand                                1
geneId                     SMESG000027261.1
transcriptId               SMEST027261001.1
distanceToTSS                        -20387
Type                                   chip
Region                           Intergenic
ATAC peak                       

In [79]:
for i in range(len(output_df_FACS)):
    df = FACS_prop[FACS_prop["gene"] == output_df_FACS.iloc[i]["geneId"]]
    if len(df) > 0:
        output_df_FACS.iloc[i, 26] = df.iloc[0,5]
        output_df_FACS.iloc[i, 27] = df.iloc[0,1]
# output_df_FACS

In [80]:
output_df_FACS.to_csv("/Users/edwinlock/Desktop/Divya/Thesis/enhancers/enhancer_annotation_FACS.csv", index=False)

## Commands for plotting figures

Copy and paste these commands into R.

In [81]:
print(upSetR_cmd(d, sets, "dualsummits.pdf"))

library(UpSetR)
expressionInput <- c(
`Intronic region&ATAC peak` = 54,
`Intergenic region&ATAC peak` = 121,
`Intronic region&H3K4me3` = 350,
`Intergenic region&H3K4me3` = 1074,
`Intergenic region&ATAC peak&RNA transcript` = 118,
`Intronic region&Accessible region` = 1085,
`Intergenic region&Accessible region` = 1605,
`Intergenic region&ATAC peak&RNA transcript&H3K4me3` = 66,
`Intronic region&Accessible region&H3K4me3` = 281,
`Intergenic region&Accessible region&H3K4me3` = 867,
`Intronic region&ATAC peak&H3K4me3` = 12,
`Intergenic region&ATAC peak&H3K4me3` = 67,
`Intergenic region&RNA transcript` = 2126,
`Intergenic region&Accessible region&RNA transcript` = 1505)
data <- fromExpression(expressionInput)
sets <- c('Intronic region', 'Intergenic region', 'Accessible region', 'ATAC peak', 'RNA transcript', 'H3K4me3')
pdf(file="dualsummits.pdf")
upset(data, sets=sets,
 order.by=c("degree", "freq"),
 main.bar.color="gray23",
 matrix.color="gray23",
 text.scale=1.3)
dev.off()



In [82]:
print(upSetR_cmd(d_k4, sets_k4, "unique_h3k4me1.pdf", colour="slateblue4"))

library(UpSetR)
expressionInput <- c(
`Intronic region&ATAC peak` = 118,
`Intergenic region&ATAC peak` = 207,
`Intronic region&H3K4me3` = 1054,
`Intergenic region&H3K4me3` = 1618,
`Intergenic region&ATAC peak&RNA transcript` = 197,
`Intronic region&Accessible region` = 2279,
`Intergenic region&Accessible region` = 2649,
`Intergenic region&ATAC peak&RNA transcript&H3K4me3` = 53,
`Intronic region&Accessible region&H3K4me3` = 704,
`Intergenic region&Accessible region&H3K4me3` = 1152,
`Intronic region&ATAC peak&H3K4me3` = 33,
`Intergenic region&ATAC peak&H3K4me3` = 54,
`Intergenic region&RNA transcript` = 3800,
`Intergenic region&Accessible region&RNA transcript` = 2561)
data <- fromExpression(expressionInput)
sets <- c('Intronic region', 'Intergenic region', 'Accessible region', 'ATAC peak', 'RNA transcript', 'H3K4me3')
pdf(file="unique_h3k4me1.pdf")
upset(data, sets=sets,
 order.by=c("degree", "freq"),
 main.bar.color="slateblue4",
 matrix.color="slateblue4",
 text.scale=1.3)
dev.off()



In [83]:
print(upSetR_cmd(d_k27, sets_k27, "unique_h3k27ac.pdf", colour="red4"))

library(UpSetR)
expressionInput <- c(
`Intronic region&ATAC peak` = 257,
`Intergenic region&ATAC peak` = 642,
`Intronic region&H3K4me3` = 4995,
`Intergenic region&H3K4me3` = 3967,
`Intergenic region&ATAC peak&RNA transcript` = 94,
`Intronic region&Accessible region` = 5153,
`Intergenic region&Accessible region` = 11010,
`Intergenic region&ATAC peak&RNA transcript&H3K4me3` = 72,
`Intronic region&Accessible region&H3K4me3` = 2394,
`Intergenic region&Accessible region&H3K4me3` = 2599,
`Intronic region&ATAC peak&H3K4me3` = 104,
`Intergenic region&ATAC peak&H3K4me3` = 143,
`Intergenic region&RNA transcript` = 2074,
`Intergenic region&Accessible region&RNA transcript` = 1497)
data <- fromExpression(expressionInput)
sets <- c('Intronic region', 'Intergenic region', 'Accessible region', 'ATAC peak', 'RNA transcript', 'H3K4me3')
pdf(file="unique_h3k27ac.pdf")
upset(data, sets=sets,
 order.by=c("degree", "freq"),
 main.bar.color="red4",
 matrix.color="red4",
 text.scale=1.3)
dev.off()

